In [11]:
%reload_ext autoreload
%autoreload 2
import numpy as np, sys, os, pandas as pd,json, os
sys.path.insert(1, '../../')
from getting_data import load_sample, feature_key_list, categorical_feature_key_list
from s2search_score_pipelining import get_scores
from s2search_score_pdp import save_pdp_to_npz

setting = 0
exp_name = 'exp5'
sample_name = 'cslg'

query = 'Machine Learning'

categorical_name = {}

def remove_duplicate(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

categorical_name_file = os.path.join('.', 'scores', f'{sample_name}_categorical_name.npz')

if not os.path.exists(categorical_name_file):
    # get categorical_name
    for i in range(len(feature_key_list)):
        feature_name = feature_key_list[i]
        if feature_name in categorical_feature_key_list:
            df = load_sample(exp_name, sample_name, query=query, sort=feature_name, rank_f=get_scores)
            if feature_name == 'authors':
                l = [json.dumps(x) for x in df[feature_name]]
            else:
                l = list(df[feature_name])
            categorical_name[i] = remove_duplicate(l)
            
    save_pdp_to_npz('.', categorical_name_file, title=categorical_name[0], abstract=categorical_name[1], \
    venue=categorical_name[2], authors=categorical_name[3])
else:
    load = np.load(categorical_name_file)
    categorical_name[0] = load['title']
    categorical_name[1] = load['abstract']
    categorical_name[2] = load['venue']
    categorical_name[3] = load['authors']

In [12]:
df = load_sample(exp_name, sample_name)
paper_data = json.loads(df.to_json(orient='records'))

y_pred_file = os.path.join('.', 'scores', f'{sample_name}_y_pred.npz')
if not os.path.exists(y_pred_file):
    y_pred = get_scores(query, paper_data)
    save_pdp_to_npz('.', y_pred_file, y_pred)
else:
    y_pred = np.load(y_pred_file)['arr_0']

In [13]:
def get_class(score):
    if score <= -17:
        return '(,-17]'
    elif score <= -10:
        return '(-17, -10]'
    elif score <= -5:
        return '(-10, -5]'    
    elif score <= 0:
        return '(-5, <0]'  
    elif score <= 3:
        return '(0, 3]'
    elif score <= 5:
        return '(3, 5]'
    elif score <= 6:
        return '(5, 6]'
    elif score <= 7:
        return '(6, 7]'
    elif score <= 8:
        return '(7, 8]'
    elif score <= 9:
        return '(8, 9]'
    else:
        return '(9,)'

# make class_name
class_name = ['(,-17]','(-17, -10]','(-10, -5]','(-5, <0]','(0, 3]','(3, 5]','(5, 6]','(6, 7]','(7, 8]','(8, 9]','(9,)']

categorical_name_map = {}

for i in range(len(feature_key_list)):
    feature_name = feature_key_list[i]
    if feature_name in categorical_feature_key_list:
        categorical_name_map[i] = {}
        values = categorical_name[i]
        for j in range(len(values)):
            value = values[j]
            categorical_name_map[i][value] = j

# encoding data
for i in range(len(paper_data)):
    paper_data[i] = [
        categorical_name_map[0][paper_data[i]['title']], categorical_name_map[1][paper_data[i]['abstract']],
        categorical_name_map[2][paper_data[i]['venue']], categorical_name_map[3][json.dumps(paper_data[i]['authors'])],
        paper_data[i]['year'],
        paper_data[i]['n_citations']
    ]

In [14]:
def decode_paper(encoded_p):
    return dict(
        title=categorical_name[0][encoded_p[0]],
        abstract=categorical_name[1][encoded_p[1]],
        venue=categorical_name[2][encoded_p[2]],
        authors=json.loads(categorical_name[3][encoded_p[3]]),
        year=encoded_p[4],
        n_citations=encoded_p[5],
    )

decoded_i = 92937

print(paper_data[decoded_i])
print(df.iloc[decoded_i])
print(decode_paper(paper_data[decoded_i]))

[55397, 28139, 2995, 84492, 2019, 61]
title          Pricing options and computing implied volatili...
abstract       This paper proposes a data-driven approach, by...
venue                                                      Risks
authors        [Shuaiqiang  Liu, Cornelis W. Oosterlee, Sande...
year                                                        2019
n_citations                                                   61
Name: 92937, dtype: object
{'title': 'Pricing options and computing implied volatilities using neural networks', 'abstract': 'This paper proposes a data-driven approach, by means of an Artificial Neural Network (ANN), to value financial options and to calculate implied volatilities with the aim of accelerating the corresponding numerical methods. With ANNs being universal function approximators, this method trains an optimized ANN on a data set generated by a sophisticated financial model, and runs the trained ANN as an agent of the original solver in a fast and eff

In [15]:
from anchor import anchor_tabular

paper_data = np.array(paper_data)

explainer = anchor_tabular.AnchorTabularExplainer(
    class_name,
    feature_key_list,
    paper_data,
    categorical_name)

In [16]:
def pred_fn(x):
    predictions = get_scores(query, [decode_paper(p) for p in x], pt = False)
    encoded_pred = [class_name.index(get_class(pp)) for pp in predictions]
    return np.array(encoded_pred)

idx = 9999
exp = explainer.explain_instance(paper_data[idx], pred_fn, threshold=0.95)

Loading process ranker model...
Load the process s2 ranker within 10.85 sec
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 100 scores with worker 66189
ge

In [17]:
print('Prediction: ', explainer.class_names[pred_fn([paper_data[idx]])[0]])
print('-------------------')
print('Anchor: \n%s' % ('\nAND\n'.join(exp.names())))
print('\nPrecision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

get ranker in 66189 with global setting: True and gb_ranker len 1
[Main taks:0] compute 1 scores with worker 66189
Prediction:  (-10, -5]
-------------------
Anchor: 
abstract = In this paper, a novel unsupervised low-rank representation model, i.e., Auto-weighted Low-Rank Representation (ALRR), is proposed to construct a more favorable similarity graph (SG) for clustering. In particular, ALRR enhances the discriminability of SG by capturing the multi-subspace structure and extracting the salient features simultaneously. Specifically, an auto-weighted penalty is introduced to learn a similarity graph by highlighting the effective features, and meanwhile, overshadowing the disturbed features. Consequently, ALRR obtains a similarity graph that can preserve the intrinsic geometrical structures within the data by enforcing a smaller similarity on two dissimilar samples. Moreover, we employ a block-diagonal regularizer to guarantee the learned graph contains k diagonal blocks. This can faci

In [24]:
import datetime
import pytz
utc_tz = pytz.timezone('America/Montreal')

metrics = dict(
    title=0,
    abstract=0,
    venue=0,
    authors=0,
    year=0,
    n_citations=0,
)

for i in range(len(paper_data)):
    print(i, datetime.datetime.now(tz=utc_tz).strftime("%m/%d/%Y, %H:%M:%S"))
    exp = explainer.explain_instance(paper_data[i], pred_fn, threshold=0.95)
    for name in exp.names():
        for feature_name in metrics.keys():
            if name.startswith(f'{feature_name} = '):
                metrics[feature_name] += 1

metrics

0 03/12/2022, 12:53:23
1 03/12/2022, 12:53:25
2 03/12/2022, 12:53:27
3 03/12/2022, 12:53:29
4 03/12/2022, 12:53:31
5 03/12/2022, 12:53:33
6 03/12/2022, 12:53:36
7 03/12/2022, 12:53:41
8 03/12/2022, 12:53:42
9 03/12/2022, 12:53:44
10 03/12/2022, 12:53:47
11 03/12/2022, 12:53:50
12 03/12/2022, 12:53:53
13 03/12/2022, 12:53:55
14 03/12/2022, 12:53:57
15 03/12/2022, 12:53:58
16 03/12/2022, 12:54:00
17 03/12/2022, 12:54:02
18 03/12/2022, 12:54:04
19 03/12/2022, 12:54:05
20 03/12/2022, 12:54:07
21 03/12/2022, 12:54:18
22 03/12/2022, 12:54:31
23 03/12/2022, 12:54:33
24 03/12/2022, 12:54:36
25 03/12/2022, 12:54:38
26 03/12/2022, 12:54:40
27 03/12/2022, 12:54:41
28 03/12/2022, 12:54:43
29 03/12/2022, 12:54:45
30 03/12/2022, 12:54:47
31 03/12/2022, 12:54:49
32 03/12/2022, 12:54:50
33 03/12/2022, 12:54:53
34 03/12/2022, 12:54:55
35 03/12/2022, 12:54:57
36 03/12/2022, 12:54:59
37 03/12/2022, 12:55:01
38 03/12/2022, 12:55:30
39 03/12/2022, 12:55:32
40 03/12/2022, 12:55:34
41 03/12/2022, 12:55:36
42